In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
!tar xf dtd-r1.0.1.tar.gz

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random as rnd
import secrets

In [ ]:
kkp = glob.glob('./data_ori/*/k*.png')
tng = glob.glob('./data_ori/*/ten*.png')
tlr = glob.glob('./data_ori/*/tel*.png')

In [ ]:
dtd = glob.glob('./dtd/images/*/*.jpg')

In [ ]:
len(dtd)

In [ ]:
gmb = cv2.imread(kkp[1], cv2.IMREAD_UNCHANGED)
gmb = cv2.cvtColor(gmb, cv2.COLOR_BGRA2RGBA)
gmb_a = gmb[:,:,-1]

In [ ]:
plt.imshow(gmb_a)

In [ ]:
def find_contour(gmb_a):
    """Mencari kontour dari alpha mask"""
    gmb_a = np.ascontiguousarray(gmb_a, dtype=np.uint8)

    _, contours, _ = cv2.findContours(gmb_a, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Approximate contours to polygons + get bounding rects and circles
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])

    drawing = np.zeros((gmb_a.shape[0], gmb_a.shape[1], 3), dtype=np.uint8)

    # Draw polygonal contour + bonding rects + circles
    border = []
    for i in range(len(contours)):
        if boundRect[i][3] != 1:
            border.append(boundRect[i])
        #cv2.drawContours(drawing, contours_poly, i, color)
        cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
              (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), (100,200,100), 8)
        #cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)

    border=border[0]
    x = border[2] // 2 + border[0]
    y = border[3] // 2 + border[1]

    drawing = cv2.circle(drawing, (x,y), 5, (100,200,100),5)
    
    return drawing, (x,y,border[2],border[3])

In [ ]:
a, d = find_contour(gmb_a)
plt.imshow(a)
print(d)

In [ ]:
def flip(img):
    if bool(rnd.getrandbits(1)):
        img = cv2.flip(img,1)
    return img

def imresize(img,x,y):
    img = cv2.resize(img,(x,y))
    return img

def imrotate(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

def immove(img, x, y):
    a = img
    M = np.float32([[1,0,x],[0,1,y]])
    a = cv2.warpAffine(a,M,(a.shape[0],a.shape[1]))
    return a

def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [ ]:
def generate(imlist, dtd, total = 0, gdir='./datagen/'):
            
        szw = float("{0:.2f}".format(rnd.uniform(0.8,1.2)))
        szh = float("{0:.2f}".format(rnd.uniform(0.8,1.2)))
        top = cv2.imread(secrets.choice(imlist), cv2.IMREAD_UNCHANGED)
        top = cv2.cvtColor(top, cv2.COLOR_BGRA2RGBA)
        height, width, depth = top.shape
        hi = int(height * szh)
        wi = int(width * szw)
        top = imresize(top, hi, wi)
        top = flip(top)
        top_a = top[:,:,-1]
        
       
        bottom = cv2.imread(secrets.choice(dtd))
        bottom = cv2.cvtColor(bottom, cv2.COLOR_BGR2RGB)
        bottom = imresize(bottom, 900, 900)
        
        rotd = rnd.randint(0,180)

        top = imrotate(top, rotd)
        top_a = imrotate(top_a, rotd)
        
        blank = np.zeros((bottom.shape[0], bottom.shape[1], 3), dtype=np.uint8)
        dd = np.ones(top.shape)*255
        
        x1 = rnd.randint(0,50)
        y1 = rnd.randint(0,50)
        
        rslt = overlay_transparent(bottom, top, x1, y1)

        
        cont, cord = find_contour(top_a)
        mask = overlay_transparent(blank, cont, x1, y1)

        return rslt, mask , cord

In [ ]:
rr, mm , crd = generate(tng,dtd,1)
print(crd)

In [ ]:
aa = rr+mm

In [ ]:
plt.imshow(rr)

In [ ]:
def create(data, dtg, size=100, dir='./dataval/' , prefix='file-'):
    """
    Membuat database dari sebuah gambar dengan alpha, taruh direktori di dir
    dir: lokasi penyimpanan
    prefix: menambahkan prefix pada file.
    """
    for i in range(0,size):
        if i % 100 == 0:
            print('.', end="")
        pic, _ , cord = generate(data,dtg)

        fld = dir + prefix + "{0:06d}".format(i) + '.jpg'

        dd = lambda x: f"0 " + str(((x[2]-x[0]) // 2) + x[0]) + " " + str(((x[3]-x[1]) // 2) + x[1]) + " " + \
            str(((x[2]-x[0]) // 2) + x[0]) + " " + str(((x[3]-x[1]) // 2) + x[1])
        pic = cv2.cvtColor(pic, cv2.COLOR_RGB2BGR)
        cv2.imwrite(fld, pic)

        fln = dir + prefix + "{0:06d}".format(i) + '.txt'

        with open(fln,'w') as f:
            f.write(dd(cord))

In [ ]:
def buildlist():
    flist = glob.glob('./dataval/*.jpg')
    flist.sort()
    for idx, i in enumerate(flist):
        fn = i.replace('\\','/')
        flist[idx] = fn
    
    with open('./dataval.txt','w') as f:
        for i in flist:
            f.write(i+'\n')

In [ ]:
buildlist()